### Importing the packages

In [10]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [23]:
clstr = Cluster()
session = clstr.connect()

NoHostAvailable: ('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})

In [12]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Create a keyspace 

In [13]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [14]:
import pandas as pd

# Define the sample data
data = {
    'SKU': [101, 102, 103, 104, 105, 106, 107, 108],
    'name': ['Widget A', 'Widget B', 'Widget C', 'Widget D', 'Gadget X', 'Gadget Y', 'Tool Z', 'Tool A'],
    'description': [
        'Small Widget for everyday use',
        'Medium Widget for various applications',
        'Large Widget for heavy-duty tasks',
        'Extra-Large Widget for industrial use',
        'High-Tech Gadget with advanced features',
        'Advanced Gadget with cutting-edge tech',
        'Multipurpose Tool for DIY enthusiasts',
        'Versatile Tool for professionals'
    ],
    'warehouse_num': [123, 456, 123, 789, 456, 123, 789, 123]
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


   SKU      name                              description  warehouse_num
0  101  Widget A            Small Widget for everyday use            123
1  102  Widget B   Medium Widget for various applications            456
2  103  Widget C        Large Widget for heavy-duty tasks            123
3  104  Widget D    Extra-Large Widget for industrial use            789
4  105  Gadget X  High-Tech Gadget with advanced features            456
5  106  Gadget Y   Advanced Gadget with cutting-edge tech            123
6  107    Tool Z    Multipurpose Tool for DIY enthusiasts            789
7  108    Tool A         Versatile Tool for professionals            123


In [16]:
# Create a table for inventory information
session.execute("""
    CREATE TABLE IF NOT EXISTS m14.inventory (
        SKU INT PRIMARY KEY,
        name TEXT,
        description TEXT,
        warehouse_num INT
    )
""")

In [20]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ({row[0]}, '{row[1]}', '{row[2]}', {row[3]});
        """
       )


        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (101, Widget A, Small Widget for everyday use, 123);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (102, Widget B, Medium Widget for various applications, 456);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (103, Widget C, Large Widget for heavy-duty tasks, 123);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (104, Widget D, Extra-Large Widget for industrial use, 789);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (105, Gadget X, High-Tech Gadget with advanced features, 456);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (106, Gadget Y, Advanced Gadget with cutting-edge tech, 123);
        

        INSERT IN

In [21]:
# Create an index on warehouse_num
session.execute("CREATE INDEX IF NOT EXISTS warehouse_index ON m14.inventory (warehouse_num)")


In [22]:
# Search for all products located at a given location (e.g., warehouse_num = 123)
result = session.execute("SELECT * FROM m14.inventory WHERE warehouse_num = %s", (123,))

# Print the results
print("Products located at warehouse_num 123:")
for row in result:
    print(f"SKU: {row.sku}, Name: {row.name}, Description: {row.description}, Warehouse: {row.warehouse_num}")


Products located at warehouse_num 123:
SKU: 108, Name: Tool A, Description: Versatile Tool for professionals, Warehouse: 123
SKU: 106, Name: Gadget Y, Description: Advanced Gadget with cutting-edge tech, Warehouse: 123
SKU: 101, Name: Widget A, Description: Small Widget for everyday use, Warehouse: 123
SKU: 103, Name: Widget C, Description: Large Widget for heavy-duty tasks, Warehouse: 123
